In [1]:
import sys
import os
from dotenv import load_dotenv
import pandas as pd

# プロジェクトルートのパスを追加（quantechiaがあるディレクトリ）
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
load_dotenv()

True

In [2]:
import pandas as pd
import numpy as np
from backtest.engine import BacktestEngine
from strategies.trend_following import trend_following
from data.data_fetcher import FinancialDataFetcher

In [24]:
import pandas as pd
import numpy as np

def calculate_returns(rtn_data, weight_data, shift_num=1, cost=True, cost_unit=0.0005) -> pd.DataFrame:
    """
    リターンを計算する。


    Returns:
        pd.DataFrame: リターンデータ。
    """
    if cost:
        # コストを考慮したリターンを計算
        cost_data = weight_data.shift(shift_num).diff() * cost_unit
        returns = weight_data.shift(shift_num) * (rtn_data - cost_data)
    else:
    
        returns = weight_data.shift(shift_num) * rtn_data
    

    return returns, pd.DataFrame(returns.sum(axis=1))

def calculate_portfolio(returns: pd.DataFrame, initial_capital: float) -> pd.DataFrame:
    """
    ポートフォリオを計算する。

    Args:
        returns (pd.DataFrame): リターンデータ。
        initial_capital (float): 初期資本。

    Returns:
        pd.DataFrame: ポートフォリオデータ。
    """
    # ポートフォリオの価値を計算
    if isinstance(returns, pd.Series):
        portfolio = (1 + returns).cumprod() * initial_capital
    else:
        portfolio = (1 + returns.sum(axis=1)).cumprod() * initial_capital

    return portfolio

    
def calculate_sharpe_ratio(returns: pd.DataFrame, risk_free_rate: float = 0.0) -> float:
    """
    シャープレシオを計算する。

    Args:
        returns (pd.DataFrame): リターンデータ。
        risk_free_rate (float): リスクフリーレート。

    Returns:
        float: シャープレシオ。
    """
    # 平均リターンを計算
    mean_return = returns.mean().sum()

    # リターンの標準偏差を計算
    std_dev = returns.std().sum()

    # シャープレシオを計算
    sharpe_ratio = (mean_return - risk_free_rate) / std_dev

    return sharpe_ratio

def calculate_max_drawdown( portfolio: pd.Series) -> float:
    """
    最大ドローダウンを計算する。

    Args:
        portfolio (pd.Series): ポートフォリオデータ。

    Returns:
        float: 最大ドローダウン。
    """
    # 累積最大値を計算
    cumulative_max = portfolio.cummax()

    # ドローダウンを計算
    drawdown = portfolio / cumulative_max - 1

    # 最大ドローダウンを計算
    max_drawdown = drawdown.min()

    return max_drawdown

def calculate_winning_rate( returns: pd.DataFrame) -> float:
    """
    勝率を計算する。

    Args:
        returns (pd.DataFrame): リターンデータ。

    Returns:
        float: 勝率。
    """
    # 1日ごとのリターンがプラスの日を数える
    winning_days = (returns.sum(axis=1) > 0).sum()

    # 全体の取引日数を計算
    total_days = len(returns)

    # 勝率を計算
    winning_rate = winning_days / total_days

    return winning_rate

class BacktestEngine:
    def __init__(self, price_data: pd.DataFrame, weight_data: pd.DataFrame, initial_capital: float = 1):
        """
        バックテストエンジン。

        Args:
            price_data (pd.DataFrame): 価格データ。各列が銘柄、各行が日付。
            weight_data (pd.DataFrame): ウェイトデータ。各列が銘柄、各行が日付。
            trade_units (pd.Series): 各銘柄の取引単位。
            initial_capital (float): 初期資本。
            **kwargs: 戦略関数の引数。
        """
        self.initial_capital = initial_capital
        self.price_data = price_data
        self.weight_data = weight_data
        self.portfolio = None
        self.returns = None
        self.rtn_data = self.price_data.pct_change()

    def run(self, shift_num=1, cost=True, cost_unit=0.0005):
        """
        バックテストを実行する。
        """
       
        # リターンを計算
        self.returns_by_asset, self.returns = calculate_returns(self.rtn_data, self.weight_data, shift_num, cost, cost_unit)

        # ポートフォリオを計算
        self.portfolio = calculate_portfolio(self.returns_by_asset, self.initial_capital)

    
    def evaluate(self) -> dict:
        """
        パフォーマンスを評価する。
        """
        # シャープレシオを計算
        sharpe_ratio = calculate_sharpe_ratio(self.returns)

        # 最大ドローダウンを計算
        max_drawdown = calculate_max_drawdown(self.portfolio)

        # 勝率を計算
        winning_rate = calculate_winning_rate(self.returns)

        return {
            "sharpe_ratio": sharpe_ratio,
            "max_drawdown": max_drawdown,
            "winning_rate": winning_rate,
        }





In [29]:
# 株価データを取得
fetcher = FinancialDataFetcher()
tickers = ['AAPL', 'MSFT', 'GOOG']
start_date = '2023-01-01'
end_date = '2023-03-31'
historical_data = fetcher.get_historical_data("data_reader", name=tickers, data_source='stooq')
price_data = historical_data['Close']
weight_data = trend_following(price_data, window=20)


In [30]:
# バックテストエンジンを初期化
engine = BacktestEngine(
    price_data=price_data,
    weight_data=weight_data,
    initial_capital=100000
)

In [31]:
# バックテストを実行
engine.run()
engine.evaluate()

{'sharpe_ratio': np.float64(-0.029270443224872995),
 'max_drawdown': np.float64(-0.8446642624838003),
 'winning_rate': np.float64(0.2659235668789809)}

In [ ]:

import quantstats as qs
engine.returns.columns =['Strategy']
qs.reports.html(engine.returns['Strategy'], output='report.html')


TypeError: Index(...) must be called with a collection of some kind, 'Strategy' was passed

In [ ]:
engine.returns.columns=

,0
Date,
2025-05-15,0.0
2025-05-14,0.0
2025-05-13,0.0
2025-05-12,0.0
2025-05-09,0.0
...,...
2020-05-22,0.0
2020-05-21,0.0
2020-05-20,0.0


In [55]:
# バックテストを実行
engine.run(cost=False)
engine.evaluate()

{'sharpe_ratio': np.float64(0.07754032509225241),
 'max_drawdown': np.float64(-0.2407768335908248),
 'winning_rate': np.float64(0.2786885245901639)}

In [38]:
units = (engine.weight_data.shift(1) * engine.initial_capital / (engine.price_data + 1e-6) / engine.trade_units).round()
positions = units * engine.trade_units
trade_returns = positions * engine.rtn_data
returns = trade_returns.sum(axis=1) / engine.initial_capital
engine.positions = positions

In [41]:
units

,AAPL,GOOG,MSFT
Date,,,
2023-01-03,NaN,NaN,NaN
2023-01-04,0.0,0.0,0.0
2023-01-05,0.0,0.0,0.0
2023-01-06,0.0,0.0,0.0
2023-01-09,0.0,0.0,0.0
...,...,...,...
2023-03-24,6.0,38.0,7.0
2023-03-27,6.0,39.0,7.0
2023-03-28,6.0,40.0,7.0


In [40]:
engine.weight_data

Ticker,AAPL,GOOG,MSFT
Date,,,
2023-01-03,0,0,0
2023-01-04,0,0,0
2023-01-05,0,0,0
2023-01-06,0,0,0
2023-01-09,0,0,0
...,...,...,...
2023-03-24,1,1,1
2023-03-27,1,1,1
2023-03-28,1,1,1


In [39]:
trade_returns

,AAPL,GOOG,MSFT
Date,,,
2023-01-03,NaN,NaN,NaN
2023-01-04,0.000000,-0.000000,-0.000000
2023-01-05,-0.000000,-0.000000,-0.000000
2023-01-06,0.000000,0.000000,0.000000
2023-01-09,0.000000,0.000000,0.000000
...,...,...,...
2023-03-24,4.983287,-1.788079,3.668232
2023-03-27,-7.376003,-27.578706,-5.226829
2023-03-28,-2.388119,-16.495246,-1.456435


In [34]:
# バックテストを実行
engine.run(return_type="trade")
engine.evaluate()

ValueError: No axis named 1 for object type Series

In [36]:
# 1日ごとのリターンがプラスの日を数える
returns = engine.returns
winning_days = (returns.sum(axis=1) > 0).sum()

# 全体の取引日数を計算
total_days = len(returns)

# 勝率を計算
winning_rate = winning_days / total_days

ValueError: No axis named 1 for object type Series

In [37]:
returns.sum(axis=1)

ValueError: No axis named 1 for object type Series

In [35]:
engine.returns

Date
2023-01-03    0.000000
2023-01-04    0.000000
2023-01-05    0.000000
2023-01-06    0.000000
2023-01-09    0.000000
                ...   
2023-03-24    0.000069
2023-03-27   -0.000402
2023-03-28   -0.000203
2023-03-29    0.000238
2023-03-30    0.000047
Length: 61, dtype: float64

In [ ]:
# 結果を表示
print("Portfolio Simple:", engine.portfolio_simple)
print("Portfolio Trade:", engine.portfolio_trade)

Portfolio Simple: 0      100000.000000\n
1      100000.000000\n
2      100000.000000\n
3      100000.000000\n
4      100000.000000\n
             ... \n
95     104819.134441\n
96     104819.134441\n
97     104819.134441\n
98     104819.134441\n
99     104819.134441\n
Length: 100, dtype: float64\n

Portfolio Trade: 0      100000.000000\n
1      100000.000000\n
2      100000.000000\n
3      100000.000000\n
4      100000.000000\n
             ... \n
95     104819.134441\n
96     104819.134441\n
97     104819.134441\n
98     104819.134441\n
99     104819.134441\n
Length: 100, dtype: float64\n